# Minneapolis, MN Simulations
8760 rows per simulation<br>
196 files (removed cell-in-cell)
Annual data and Illuminance data were exported before folder was split in two. Monthly data csv file was over the Github 100MB limit

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 0.83
elec = 0.1314

---

## Combine all csv files in directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/msp/") if f.endswith(".csv")]

#files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/msp/" + file)
    all_data = pd.concat([all_data, df])
    
#all_data

#1716960   rows, 73 columns

In [5]:
# print(all_data)
# is_NaN = all_data. isnull()
# row_has_NaN = is_NaN. any(axis=1)
# rows_with_NaN = all_data[row_has_NaN]
# print(rows_with_NaN)

In [6]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]

#all_data1

In [7]:
all_data1 = all_data1.dropna().reset_index(drop=True)

#all_data1

#1716960   rows, 73 columns

-----

## Get just the wanted data (columns)

In [8]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#1716960  rows, 6 columns

-----

## Get annual energy data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [9]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [10]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [11]:
# Drop columns
get_annual.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get annual cost data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variable needs to be defined above

In [12]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [13]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanEnergy($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingEnergy($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingEnergy($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [14]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine annual energy and cost data
Combine two dfs<br>
Group by scenario <br>
Export combine file

In [15]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs, how="inner" )

#allAnnualData

In [16]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanEnergy($)"] + allAnnualData["HeatingEnergy($)"] + allAnnualData["CoolingEnergy($)"]
allAnnualData["AnnualCost($)"] = totalCost

allAnnualData

,Scenario,Date_Time,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($),AnnualEnergy(kBtu),AnnualCost($)
0,msp_102_2011_1VB-15,01/01 01:00:00,0.409048,26.402377,0.0,0.015752,0.219140,0.0,26.811425,0.234892
1,msp_102_2011_1VB-15,01/01 02:00:00,0.457762,29.546639,0.0,0.017628,0.245237,0.0,30.004400,0.262865
2,msp_102_2011_1VB-15,01/01 03:00:00,0.468867,30.263401,0.0,0.018056,0.251186,0.0,30.732268,0.269242
3,msp_102_2011_1VB-15,01/01 04:00:00,0.467571,30.179752,0.0,0.018006,0.250492,0.0,30.647323,0.268498
4,msp_102_2011_1VB-15,01/01 05:00:00,0.476243,30.739511,0.0,0.018340,0.255138,0.0,31.215754,0.273478
...,...,...,...,...,...,...,...,...,...,...
1716955,msp_2011_BBG-VB75_2159,12/31 20:00:00,0.275373,17.774214,0.0,0.010605,0.147526,0.0,18.049588,0.158130
1716956,msp_2011_BBG-VB75_2159,12/31 21:00:00,0.274240,17.701075,0.0,0.010561,0.146919,0.0,17.975315,0.157480
1716957,msp_2011_BBG-VB75_2159,12/31 22:00:00,0.283972,18.329204,0.0,0.010936,0.152132,0.0,18.613176,0.163068
1716958,msp_2011_BBG-VB75_2159,12/31 23:00:00,0.296788,19.156437,0.0,0.011429,0.158998,0.0,19.453225,0.170428


---
## Get scenario names 
Get the names to a list<br>
Export to file

In [17]:
# Create new df of just scenario names
# Export as a csv for javascript filter
scenarios = allAnnualData["Scenario"].unique()
print(scenarios)

['msp_102_2011_1VB-15' 'msp_102_2011_1VB-30' 'msp_102_2011_1VB-45'
 'msp_102_2011_1VB-60' 'msp_102_2011_1VB-75' 'msp_102_2011_1VB0'
 'msp_102_2011_1VB15' 'msp_102_2011_1VB30' 'msp_102_2011_1VB45'
 'msp_102_2011_1VB60' 'msp_102_2011_1VB75' 'msp_102_2011_base'
 'msp_102_2011_cellularLT' 'msp_102_2011_cellularHT'
 'msp_102_2011_1VB-15-a' 'msp_102_2011_1VB-30-a' 'msp_102_2011_1VB-45-a'
 'msp_102_2011_1VB-60-a' 'msp_102_2011_1VB-75-a' 'msp_102_2011_1VB0-a'
 'msp_102_2011_1VB15-a' 'msp_102_2011_1VB30-a' 'msp_102_2011_1VB45-a'
 'msp_102_2011_1VB60-a' 'msp_102_2011_1VB75-a' 'msp_102_2011_base-a'
 'msp_102_2011_cellularLT-a' 'msp_102_2011_cellularHT-a'
 'msp_102_BBG-VB-15_2011-a' 'msp_102_BBG-VB-30_2011-a'
 'msp_102_BBG-VB-45_2011-a' 'msp_102_BBG-VB-60_2011-a'
 'msp_102_BBG-VB-75_2011-a' 'msp_102_BBG-VB0_2011-a'
 'msp_102_BBG-VB15_2011-a' 'msp_102_BBG-VB30_2011-a'
 'msp_102_BBG-VB45_2011-a' 'msp_102_BBG-VB60_2011-a'
 'msp_102_BBG-VB75_2011-a' 'msp_102_102_base' 'msp_102_102_1VB-15'
 'msp_102_10

In [18]:
scenarios.tofile("data/final/msp_scenarios.csv", sep=",", format="%s")

---
## Group and export
Group dataframe by scenario<br>
Export to file

In [19]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"])

annualData.sum().round(2)

,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($),AnnualEnergy(kBtu),AnnualCost($)
Scenario,,,,,,,,
msp_102_102_1VB-15,2523.82,105536.02,9348.32,97.19,875.95,360.00,117408.16,1333.14
msp_102_102_1VB-30,2521.46,105458.37,9335.04,97.10,875.30,359.49,117314.88,1331.89
msp_102_102_1VB-45,2505.36,106669.40,8983.71,96.48,885.36,345.96,118158.47,1327.79
msp_102_102_1VB-60,2468.30,108967.99,8249.86,95.05,904.43,317.70,119686.15,1317.19
msp_102_102_1VB-75,2409.84,112012.32,7173.16,92.80,929.70,276.24,121595.33,1298.74
...,...,...,...,...,...,...,...,...
msp_2011_BBG-VB60_2159-a,2122.93,108767.25,4713.05,81.75,902.77,181.50,115603.23,1166.02
msp_2011_BBG-VB75_102,2203.47,114541.59,4624.24,84.85,950.70,178.08,121369.30,1213.63
msp_2011_BBG-VB75_102-a,2144.85,110444.39,4671.32,82.60,916.69,179.89,117260.56,1179.18


In [20]:
# Export grouped file to csv
annualData.sum().round(2).to_csv("data/final/msp_annual_data.csv", header=True, index=True)

In [21]:
# # Export grouped file to json
# annualData.sum().to_json("data/final/msp_annual_data.json")

-----

## Get monthly energy data
Group by scenario <br>
Groups by month <br>
Change units from Joules to therms and kWh

In [22]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [23]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [24]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh)
0,msp_102_2011_1VB-15,01/01 01:00:00,0.0,0.119880,0.264024,0.0
1,msp_102_2011_1VB-15,01/01 02:00:00,0.0,0.134157,0.295466,0.0
2,msp_102_2011_1VB-15,01/01 03:00:00,0.0,0.137411,0.302634,0.0
3,msp_102_2011_1VB-15,01/01 04:00:00,0.0,0.137031,0.301798,0.0
4,msp_102_2011_1VB-15,01/01 05:00:00,0.0,0.139573,0.307395,0.0
...,...,...,...,...,...,...
1716955,msp_2011_BBG-VB75_2159,12/31 20:00:00,0.0,0.080704,0.177742,0.0
1716956,msp_2011_BBG-VB75_2159,12/31 21:00:00,0.0,0.080372,0.177011,0.0
1716957,msp_2011_BBG-VB75_2159,12/31 22:00:00,0.0,0.083224,0.183292,0.0
1716958,msp_2011_BBG-VB75_2159,12/31 23:00:00,0.0,0.086980,0.191564,0.0


## Get monthly cost data

Group by scenario <br>
Groups by month <br>
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [25]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [26]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanEnergy($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingEnergy($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingEnergy($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [27]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly_costs

---

## Combine monthly energy and cost data
Combine two dfs<br>
Create Month Column<br>
Group by scenario <br>
Export combine file

In [28]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner" )

#allMonthlyData

In [29]:
# Add energy and cost totals
totalEnergyMonthly = allMonthlyData["FanEnergy(kWh)"] + allMonthlyData["HeatingEnergy(therms)"] + allMonthlyData["CoolingEnergy(kWh)"]
allMonthlyData["MonthlyEnergy(kBtu)"] = totalEnergyMonthly

totalCostMonthly = allMonthlyData["FanEnergy($)"] + allMonthlyData["HeatingEnergy($)"] + allMonthlyData["CoolingEnergy($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

averageIlluminanceMonthly = allMonthlyData["Illuminance(lux)"].mean()

#allMonthlyData

In [30]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

allMonthlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),MonthlyEnergy(kBtu),MonthlyCost($),Month
0,msp_102_2011_1VB-15,01/01 01:00:00,0.0,0.119880,0.264024,0.0,0.015752,0.219140,0.0,0.383904,0.234892,01
1,msp_102_2011_1VB-15,01/01 02:00:00,0.0,0.134157,0.295466,0.0,0.017628,0.245237,0.0,0.429623,0.262865,01
2,msp_102_2011_1VB-15,01/01 03:00:00,0.0,0.137411,0.302634,0.0,0.018056,0.251186,0.0,0.440045,0.269242,01
3,msp_102_2011_1VB-15,01/01 04:00:00,0.0,0.137031,0.301798,0.0,0.018006,0.250492,0.0,0.438829,0.268498,01
4,msp_102_2011_1VB-15,01/01 05:00:00,0.0,0.139573,0.307395,0.0,0.018340,0.255138,0.0,0.446968,0.273478,01
...,...,...,...,...,...,...,...,...,...,...,...,...
1716955,msp_2011_BBG-VB75_2159,12/31 20:00:00,0.0,0.080704,0.177742,0.0,0.010605,0.147526,0.0,0.258446,0.158130,12
1716956,msp_2011_BBG-VB75_2159,12/31 21:00:00,0.0,0.080372,0.177011,0.0,0.010561,0.146919,0.0,0.257383,0.157480,12
1716957,msp_2011_BBG-VB75_2159,12/31 22:00:00,0.0,0.083224,0.183292,0.0,0.010936,0.152132,0.0,0.266516,0.163068,12
1716958,msp_2011_BBG-VB75_2159,12/31 23:00:00,0.0,0.086980,0.191564,0.0,0.011429,0.158998,0.0,0.278544,0.170428,12


In [31]:
# Drop Illuminance column...don't need the totals
allMonthlyData2 = allMonthlyData.drop(["Illuminance(lux)", "MonthlyEnergy(kBtu)" ], axis=1)

In [32]:
# Group by scenario and month
MonthlyData = allMonthlyData2.groupby(["Scenario", "Date_Time"])

MonthlyData.sum()

FanEnergy(kWh)  \
Scenario                 Date_Time                          
msp_102_102_1VB-15        01/01  01:00:00        0.129020   
                          01/01  02:00:00        0.144036   
                          01/01  03:00:00        0.147359   
                          01/01  04:00:00        0.146904   
                          01/01  05:00:00        0.149366   
...                                                   ...   
msp_2011_BBG-VB75_2159-a  12/31  20:00:00        0.078111   
                          12/31  21:00:00        0.077783   
                          12/31  22:00:00        0.080620   
                          12/31  23:00:00        0.084327   
                          12/31  24:00:00        0.083514   

                                           HeatingEnergy(therms)  \
Scenario                 Date_Time                                 
msp_102_102_1VB-15        01/01  01:00:00               0.284153   
                          01/01  02:00:00               0.317225   
                          01/01  03:00:00               0.324542   
                          01/01  04:00:00               0.323542   
                          01/01  05:00:00               0.328963   
...                                                          ...   
msp_2011_BBG-VB75_2159-a  12/31  20:00:00               0.172032   
                          12/31  21:00:00               0.171309   
                          12/31  22:00:00               0.177557   
                          12/31  23:00:00               0.185721   
                          12/31  24:00:00               0.183932   

                                           CoolingEnergy(kWh)  FanEnergy($)  \
Scenario                 Date_Time                                            
msp_102_102_1VB-15        01/01  01:00:00                 0.0      0.016953   
                          01/01  02:00:00                 0.0      0.018926   
                          01/01  03:00:00                 0.0      0.019363   
                          01/01  04:00:00                 0.0      0.019303   
                          01/01  05:00:00                 0.0      0.019627   
...                                                       ...           ...   
msp_2011_BBG-VB75_2159-a  12/31  20:00:00                 0.0      0.010264   
                          12/31  21:00:00                 0.0      0.010221   
                          12/31  22:00:00                 0.0      0.010593   
                          12/31  23:00:00                 0.0      0.011081   
                          12/31  24:00:00                 0.0      0.010974   

                                           HeatingEnergy($)  CoolingEnergy($)  \
Scenario                 Date_Time                                              
msp_102_102_1VB-15        01/01  01:00:00          0.235847               0.0   
                          01/01  02:00:00          0.263297               0.0   
                          01/01  03:00:00          0.269370               0.0   
                          01/01  04:00:00          0.268539               0.0   
                          01/01  05:00:00          0.273039               0.0   
...                                                     ...               ...   
msp_2011_BBG-VB75_2159-a  12/31  20:00:00          0.142786               0.0   
                          12/31  21:00:00          0.142186               0.0   
                          12/31  22:00:00          0.147373               0.0   
                          12/31  23:00:00          0.154148               0.0   
                          12/31  24:00:00          0.152663               0.0   

                                           MonthlyCost($)  
Scenario                 Date_Time                         
msp_102_102_1VB-15        01/01  01:00:00        0.252800  
                          01/01  02:00:00        0.282223  
                          01/01  03:00:00 

In [33]:
# Export grouped file to csv
MonthlyData.sum().round(2).to_csv("data/final/msp_monthly_data.csv", header=True, index=True)

In [34]:
# # Export grouped file to json, issues with exporting grouped csv
# MonthlyData.sum().round(2).to_json("data/final/msp_monthly_data.json")

In [35]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData2.groupby(["Scenario", "Month"])

MonthlyData2.sum()

FanEnergy(kWh)  HeatingEnergy(therms)  \
Scenario                 Month                                          
msp_102_102_1VB-15        01        122.654812             270.134544   
                          02         76.147665             167.707360   
                          03         52.883199             115.312696   
                          04         37.547450              63.443568   
                          05         31.699022               9.720076   
...                                        ...                    ...   
msp_2011_BBG-VB75_2159-a  08         26.980666               0.174588   
                          09         17.777222              15.171464   
                          10         26.458971              58.273148   
                          11         60.607537             133.481835   
                          12         99.718826             219.620406   

                                CoolingEnergy(kWh)  FanEnergy($)  \
Scenario                 Month                                     
msp_102_102_1VB-15        01              0.000000     16.116842   
                          02              0.000000     10.005803   
                          03              4.893978      6.948852   
                          04             90.962355      4.933735   
                          05            280.868509      4.165252   
...                                            ...           ...   
msp_2011_BBG-VB75_2159-a  08            288.626884      3.545259   
                          09            115.487212      2.335927   
                          10              0.000000      3.476709   
                          11              0.000000      7.963830   
                          12              0.000000     13.103054   

                                HeatingEnergy($)  CoolingEnergy($)  \
Scenario                 Month                                       
msp_102_102_1VB-15        01          224.211671          0.000000   
                          02          139.197109          0.000000   
                          03           95.709537          0.643069   
                          04           52.658162         11.952453   
                          05            8.067663         36.906122   
...                                          ...               ...   
msp_2011_BBG-VB75_2159-a  08            0.144908         37.925573   
                          09           12.592316         15.175020   
                          10           48.366713          0.000000   
                          11          110.789923          0.000000   
                          12          182.284937          0.000000   

                                MonthlyCost($)  
Scenario                 Month                  
msp_102_102_1VB-15        01        240.328514  
                          02        149.202912  
                          03        103.301459  
                          04         69.544350  
                          05         49.139037  
...                                        ...  
msp_2011_BBG-VB75_2159-a  08         41.615740  
                          09         30.103262  
                          10         51.843422  
                          11        118.753754  
                          12        195.387991  

[2352 rows x 7 columns]

In [36]:
# Export grouped file to csv
MonthlyData2.sum().round(2).to_csv("data/final/msp_monthly2_data.csv", header=True, index=True)

## Get monthly average illuminance

Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [42]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]
#Illuminance

In [43]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Date_Time"])

MonthlyIlluminance.mean()

Illuminance(lux)
Scenario                 Date_Time                         
msp_102_102_1VB-15        01/01  01:00:00               0.0
                          01/01  02:00:00               0.0
                          01/01  03:00:00               0.0
                          01/01  04:00:00               0.0
                          01/01  05:00:00               0.0
...                                                     ...
msp_2011_BBG-VB75_2159-a  12/31  20:00:00               0.0
                          12/31  21:00:00               0.0
                          12/31  22:00:00               0.0
                          12/31  23:00:00               0.0
                          12/31  24:00:00               0.0

[1716960 rows x 1 columns]

In [44]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("data/final/msp_monthly_illuminance.csv", header=True, index=True)

In [45]:
# # Export grouped file to json, issues with exporting grouped csv
# MonthlyIlluminance.mean().round(2).to_json("data/final/msp_monthly_illuminance.json")

In [46]:
# Group by scenario
MonthlyIlluminance2 = Illuminance.groupby(["Scenario", "Month"])

MonthlyIlluminance2.mean()

Illuminance(lux)
Scenario                 Month                  
msp_102_102_1VB-15        01         1807.783179
                          02         1231.173784
                          03         1307.430292
                          04          817.503233
                          05          831.925604
...                                          ...
msp_2011_BBG-VB75_2159-a  08           24.833176
                          09           27.156514
                          10           25.790813
                          11           19.085905
                          12           16.415345

[2352 rows x 1 columns]

In [47]:
# Export grouped file to csv
MonthlyIlluminance2.mean().round(2).to_csv("data/final/msp_monthly_illuminance2.csv", header=True, index=True)

------

## Get daily data

In [ ]:
# get_daily = all_data2.copy(deep=True)

# #get_daily

In [ ]:
# # Convert Joules to KBtu in dataframe
# get_daily["FanEnergy(kBtu)"] = get_daily["FanEnergy[J](Hourly)"] * .00000094781712
# get_daily["HeatingEnergy(kBtu)"] = get_daily["HeatingEnergy[J](Hourly)"] * .00000094781712
# get_daily["CoolingEnergy(kBtu)"] = get_daily["CoolingEnergy[J](Hourly) "] * .00000094781712

# #get_daily

In [ ]:
# # Convert Joules to kWh and therms in new dataframe
# get_daily["FanEnergy(kWh)"] = get_daily["FanEnergy[J](Hourly)"] * .000000277777778
# get_daily["HeatingEnergy(therms)"] = get_daily["HeatingEnergy[J](Hourly)"] * .0000000094781712
# get_daily["CoolingEnergy(kWh)"] = get_daily["CoolingEnergy[J](Hourly) "] * .000000277777778

# #get_daily

In [ ]:
# # Drop columns
# get_daily.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# #get_daily

In [ ]:
# daily_data = get_daily.groupby(["Scenario", "Date_Time"])

# daily_data.sum()

In [ ]:
# # Export grouped file to csv for daily energy sums
# daily_data.sum().to_csv("data/final/msp_daily_energy_sum.csv")

------

## Get daily cost data

In [ ]:
# get_daily_costs = all_data2.copy(deep=True)

# #get_daily

In [ ]:
# # Convert Joules to KBtu in dataframe and add cost multiplier
# # Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# # Elec kbtu => Kwh (kbtu * .000000277777778)
# get_daily_costs["FanEnergy($)"] = (get_daily_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
# get_daily_costs["HeatingEnergy($)"] = ((get_daily_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
# get_daily_costs["CoolingEnergy($)"] = (get_daily_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

# #get_daily_costs

In [ ]:
# # Drop columns
# get_daily_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# #get_daily_costs

In [ ]:
# get_daily_costs["Month"] = get_daily_costs.Date_Time.str.slice(0,3)

# #get_daily_costs

In [ ]:
# daily_costs = get_daily_costs.groupby(["Scenario", "Month"])

# daily_costs.sum()

In [ ]:
# # Export grouped file to csv for daily cost sums
# daily_costs.sum().to_csv("data/final/msp_daily_cost_sum.csv")